В этом задании вам необходимо будет:

1. Реализовать формулу подсчета длительности теста, сравнить ее с онлайн калькуляторами (например https://mindbox.ru/tools/ab-test-calculator/ ). При сравнении оценить мощность критерия при указанном изменении и рассчитанном количестве наблюдений в выборке. 

2. Реализовать метод линеаризации. Проверить для него корректность и мощность. Входные данные - синтетически сгенерированные.

3. Реализовать метод CUPED. Проверить для него корректность и мощность. Данные на этапе до A/B теста необходимо сгенерировать один раз, далее синтетически генерировать только часть, связанную с проведением A/B-теста.

In [1]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
from scipy import stats
from math import asin
import numpy as np

In [2]:
def calculate_sample_size_per_group(alpha, beta, p1, p2):
    """
    Рассчитываем размер выборки для каждой группы A/B теста.
    
    :param alpha: Уровень значимости (обычно 0.05)
    :param beta: Вероятность ошибки второго рода, 1 - мощность
    :param p1: Базовый уровень конверсии в контрольной группе
    :param p2: Ожидаемый уровень конверсии в экспериментальной группе после изменений
    :return: Размер выборки для каждой группы
    """
    # Стандартное отклонение для двух пропорций
    sigma = lambda p: np.sqrt(2 * p * (1 - p))
    
    # Стандартная ошибка разности между двумя пропорциями
    se = lambda p1, p2: np.sqrt(sigma(p1)**2 + sigma(p2)**2)
    
    # Рассчитываем Z-значения для альфа и бета
    z_alpha = stats.norm.ppf(1 - alpha / 2)
    z_beta = stats.norm.ppf(1 - beta)
    
    # Размер выборки для каждой группы
    n = ((z_alpha + z_beta) * se(p1, p2))**2 / (p2 - p1)**2
    return int(np.ceil(n))  # Округляем вверх до целого числа

# Зададим параметры теста
alpha = 0.05  # Уровень значимости 5%
power = 0.8   # Мощность 80%
beta = 1 - power  # Ошибка II рода

# Базовый уровень конверсии и ожидаемое улучшение
base_conversion_rate = 0.10  # 10%
minimum_detectable_effect = 0.01  # Ожидаемое улучшение на 1%

# Ожидаемый уровень конверсии после внедрения изменений
expected_conversion_rate = base_conversion_rate + minimum_detectable_effect

# Рассчитаем размер выборки для каждой группы
sample_size = calculate_sample_size_per_group(alpha, beta, base_conversion_rate, expected_conversion_rate)
print(f"Необходимый размер выборки для каждой группы: {sample_size}")

Необходимый размер выборки для каждой группы: 29497


In [3]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

# Рассчитаем мощность критерия
effect_size = proportion_effectsize(base_conversion_rate, expected_conversion_rate)
power_analysis = NormalIndPower()
power_result = power_analysis.solve_power(effect_size=effect_size, 
                                          nobs1=sample_size, 
                                          alpha=alpha)

print(f"Рассчитанная мощность: {power_result:.4f}")

Рассчитанная мощность: 0.9774


In [6]:
def calculate_total_sample_size(num_groups, sample_size_per_group):
    """
    Рассчитывает общий размер выборки, учитывая количество групп в A/B тесте.
    
    :param num_groups: Количество групп в тесте.
    :param sample_size_per_group: Размер выборки для одной группы.
    :return: Общий размер выборки для всех групп.
    """
    return num_groups * sample_size_per_group

# Количество вариантов в тесте
num_variants = 2  # Контрольная и одна тестовая группа

# Размер выборки для каждой группы, как рассчитано выше
sample_size_per_group = calculate_sample_size_per_group(alpha, beta, base_conversion_rate, expected_conversion_rate)

# Общий размер выборки для всех групп
total_sample_size = calculate_total_sample_size(num_variants, sample_size_per_group)
print(f"Общий размер выборки для всех групп: {total_sample_size}")

# Длительность теста зависит от скорости набора пользователей и не может быть рассчитана без знания этой скорости.
# Пример: если ваш сайт или приложение получает 1000 посетителей в день, вы можете оценить длительность теста:
daily_visitors = 1000
test_duration_days = total_sample_size / daily_visitors
print(f"Ориентировочная длительность теста в днях: {test_duration_days:.1f}")

Общий размер выборки для всех групп: 58994
Ориентировочная длительность теста в днях: 59.0


Сайт не расчитывает длительность теста и параметры задаются несколько иначе.

In [5]:
#import scipy.stats as stats
np.seterr(divide='ignore')

def calculate_sample_size(base_conversion_rate, minimum_detectable_effect, alpha=0.05, power=0.8):
    """
    Расчет размера выборки на основе базовой конверсии, минимального обнаруживаемого эффекта,
    уровня значимости (alpha) и мощности критерия (power).
    """
    # Значение Z-критерия для заданного уровня значимости
    z_alpha = stats.norm.ppf(1 - alpha/2)
    # Значение Z-критерия для заданной мощности
    z_power = stats.norm.ppf(power)
    
    # Преобразование базовой конверсии и МДЭ в пропорции
    p1 = base_conversion_rate
    p2 = base_conversion_rate + minimum_detectable_effect
    p_combined = (p1 + p2) / 2
    
    # Расчет размера выборки
    sample_size = (z_alpha*np.sqrt(2*p_combined*(1-p_combined)) + z_power*np.sqrt(p1*(1-p1)+p2*(1-p2)))**2 / minimum_detectable_effect**2
    return np.ceil(sample_size)  # Округляем в большую сторону

# Тестирование
base_conv_rate = 0.1  # Базовая конверсия
min_det_effect = 0.02  # Минимальный обнаруживаемый эффект (например, увеличение на 2%)
sample_size = calculate_sample_size(base_conv_rate, min_det_effect)
print(f"Размер выборки для каждой группы: {sample_size:.0f}")

# Пример функции линеаризации для пропорций
def linearize_proportions(data):
    """
    Линеаризация массива данных с пропорциями путем использования логарифмического преобразования.
    """
    return np.log(data / (1 - data))

# Генерация синтетических данных
np.random.seed(42)  # Устанавливаем seed для воспроизводимости
group_a = np.random.binomial(1, base_conv_rate, int(sample_size))
group_b = np.random.binomial(1, base_conv_rate + min_det_effect, int(sample_size))

# Линеаризация данных
linearized_a = linearize_proportions(group_a)
linearized_b = linearize_proportions(group_b)

# Проверка корректности и мощности можно провести через t-тест на линеаризированных данных
t_stat, p_value = stats.ttest_ind(linearized_a, linearized_b)
print(f"T-statistic: {t_stat}, P-value: {p_value}")

# Если P-value < alpha, мы отвергаем нулевую гипотезу о равенстве средних в двух группах.

def generate_pre_test_data(n, mean, std):
    """
    Генерация предтестовых данных, которые имитируют метрики до проведения A/B-теста.
    """
    return np.random.normal(mean, std, n)

def apply_cuped(post_test_data, pre_test_data, covariate_mean):
    """
    Применение CUPED к посттестовым данным, используя предтестовые данные.
    """
    theta = np.cov(post_test_data, pre_test_data)[0][1] / np.var(pre_test_data)
    cuped_scores = post_test_data - theta * (pre_test_data - covariate_mean)
    return cuped_scores

# Генерация предтестовых данных
pre_test_mean = 100  # Среднее значение метрики до теста
pre_test_std = 15    # Стандартное отклонение метрики до теста
pre_test_data = generate_pre_test_data(int(2*sample_size), pre_test_mean, pre_test_std)

# Генерация посттестовых данных для групп A и B
post_test_data_a = pre_test_data[:int(sample_size)] + np.random.normal(
    0, pre_test_std * 0.1, int(sample_size))
post_test_data_b = pre_test_data[int(sample_size):] + np.random.normal(
    min_det_effect, pre_test_std * 0.1, int(sample_size))

# Применение CUPED
cuped_scores_a = apply_cuped(post_test_data_a, pre_test_data[:int(sample_size)], pre_test_mean)
cuped_scores_b = apply_cuped(post_test_data_b, pre_test_data[int(sample_size):], pre_test_mean)
# Анализ CUPED-трансформированных данных
t_stat_cuped, p_value_cuped = stats.ttest_ind(cuped_scores_a, cuped_scores_b)
print(f"CUPED T-statistic: {t_stat_cuped:.4f}, P-value: {p_value_cuped:.4f}")

Размер выборки для каждой группы: 3841
T-statistic: nan, P-value: nan
CUPED T-statistic: 0.4509, P-value: 0.6521
